# Documentation

# Sample

In [1]:
import sys 
!{sys.executable} -m pip install nltk 
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install sklearn

In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

products_data = pd.read_csv("produtos.csv", delimiter=';', encoding='utf-8')

# concatenando as colunas nome e descricao
products_data['informacao'] = products_data['nome'] + products_data['descricao']
# excluindo linhas com valor de informacao ou categoria NaN
products_data.dropna(subset=['informacao', 'categoria'], inplace=True)
products_data.drop(columns=['nome', 'descricao'], inplace=True)

stop_words=set(stopwords.words("portuguese"))
# transforma a string em caixa baixa e remove stopwords
products_data['sem_stopwords'] = products_data['informacao'].str.lower().apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

tokenizer = nltk.RegexpTokenizer(r"\w+")
products_data['tokens'] = products_data['sem_stopwords'].apply(tokenizer.tokenize) # aplica o regex tokenizer
products_data.drop(columns=['sem_stopwords','informacao'],inplace=True) # Exclui as colunas antigas

products_data["strings"]= products_data["tokens"].str.join(" ") # reunindo cada elemento da lista
products_data.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/t734582/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,categoria,tokens,strings
0,livro,"[hobbit, 7ª, ed, 2013, produto, novobilbo, bol...",hobbit 7ª ed 2013 produto novobilbo bolseiro h...
1,livro,"[livro, it, coisa, stephen, king, produto, nov...",livro it coisa stephen king produto novodurant...
2,livro,"[box, crônicas, gelo, fogo, pocket, 5, livros,...",box crônicas gelo fogo pocket 5 livros produto...
3,livro,"[box, harry, potter, produto, novo, físico, sé...",box harry potter produto novo físico série har...
4,livro,"[livro, origem, dan, brown, produto, novode, o...",livro origem dan brown produto novode onde vie...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split( # Separação dos dados para teste e treino
    products_data["strings"], 
    products_data["categoria"], 
    test_size = 0.2, 
    random_state = 10
)
# CountVectorizer
vect = CountVectorizer()
vect.fit(products_data["strings"])

dataset = { 
    "X_train": vect.transform(X_train),
    "X_test": vect.transform(X_test),
    "y_train": y_train,
    "y_test" : y_test,
    "vect": vect
}

In [4]:
from sklearn.naive_bayes import MultinomialNB


clf = MultinomialNB()
clf.fit(dataset["X_train"], dataset["y_train"])

model = {
    "clf" : clf,
    "vect": dataset["vect"]
}

In [5]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
# Multinomial NB
y_prediction = model["clf"].predict(dataset["X_test"])
accuracy = accuracy_score(y_prediction, dataset["y_test"])


In [6]:
input_message = ["Figura Transformers Prime War Deluxe - E9687 - Hasbro"]

In [7]:
print(input_message)
input_message = model["vect"].transform(input_message)

['Figura Transformers Prime War Deluxe - E9687 - Hasbro']


In [8]:
final_prediction = model["clf"].predict(input_message)[0]
print("Predicted value: " + final_prediction)

Predicted value: brinquedo


In [10]:
from sklearn.naive_bayes import MultinomialNB
# Biblioteca para salvar objetos Python em disco
import pickle

clf = MultinomialNB()
clf.fit(dataset["X_train"], dataset["y_train"])

model = {
    "clf" : clf,
    "vect": dataset["vect"]
}
# Salva o modelo em um arquivo
pickle.dump(model, open("model.sav", 'wb'))